In [1]:
import tensorflow

ModuleNotFoundError: No module named 'tensorflow.python.distribute.distribution_strategy_context'

In [1]:
import MeCab
import random
import re
import requests
from bs4 import BeautifulSoup
from nltk.util import bigrams
import  neologdn
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

c:\Users\daiki\miniconda3\envs\env_lecture\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Failed to import transformers.data.data_collator because of the following error (look up to see its traceback):
No module named 'tensorflow.python.distribute.distribution_strategy_context'

## bigram

In [33]:
# 青空文庫などからデータの取得
response = requests.get("https://www.aozora.gr.jp/cards/001095/files/42621_21290.html")
response.encoding = response.apparent_encoding

# HTML からテキストデータを取得してテキスト部分を取得
soup = BeautifulSoup(response.text, "html.parser")
text = soup.get_text()
text = text.split("底本：")[0]
text = text.split("坂口安吾\n\n\n\n\r\n\u3000")[1]
text = neologdn.normalize(text)
text = re.sub("《.+?》", "", text)

In [34]:
# 分かち書き
tagger = MeCab.Tagger("-Owakati")
words = tagger.parse(text).split()

In [35]:
# bigram モデルを構築
bi_grams = list(bigrams(words))
bigram_model = {}
for bigram in bi_grams:
    if bigram[0] not in bigram_model:
        bigram_model[bigram[0]] = [bigram[1]]
    else:
        bigram_model[bigram[0]].append(bigram[1])

In [37]:
# 文章を生成する関数
def generate_sentence(model, start_word, max_length):
    # 初期単語が設定されていなければランダムに選択
    if not start_word:
        start_word = random.choice(words)
    current_word = start_word
    sentence = [current_word]
    # max_length だけ繰り返す
    for _ in range(max_length):
        # 次の単語リストを取得（無ければ終了）
        next_words = model.get(current_word)
        if not next_words:
            break
        # 単語リストからランダムに選択して追加
        next_word = random.choice(next_words)
        sentence.append(next_word)
        # 現在の単語を更新
        current_word = next_word
    return "".join(sentence)


# ある単語から始まる文章を10個生成
for i in range(10):
    generated_text = generate_sentence(bigram_model, "自分", 30)
    print(generated_text)


自分の最も薄い一本あるのはまった。白痴の由であった。天地に落下の色ではようやく終電に似た。
自分の姿であった。伊沢自身に一滴の個性だの釜だの首をしてもこの世界を数える時のためにサイパンに
自分の置き場所では、伊沢は四月十何本の小屋の穂先すら見えるの消し方が仕掛けて死ぬ、虚妄の破片の
自分の豚の空虚で、弾丸もない。ああ、結局娘を表わすから腹と、要するに如何なるアングルによって縛りつけられ、もとよりそれは
自分自身に身をねせてあり、女の家では考えた。ひと思いに、人間だの学校の国民酒場を払ってすら
自分の無限のあるのふちに法が通勤に旅立った。この煙草に演説を感覚の恥辱であった。違った。街角
自分の方が一年昔の場合にすぎないこの女が最後の本質的なものであった、明日と消え失せ、ともかく芸人
自分自身が火の出口に、いっしょに関係が、同時に起ったが火も、肺病の賤業中という言外の人はこの差し
自分自身の煙草屋が一本ずつビールを歩きかけると何食わぬ顔をむさぼり見た。え?なんと老人の前
自分自身のは度の違いそうだった。全てをふかし、それは連日賭場が白んでいる女を逃げだすためであり、私


## BERT

In [ ]:
import MeCab
import requests
from bs4 import BeautifulSoup

# トークナイザーとモデルの準備
tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt2-medium")
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")

# 青空文庫などからデータの取得
response = requests.get("https://...")
response.encoding = response.apparent_encoding

# HTMLからテキストデータを取得してテキスト部分を取得
soup = BeautifulSoup(response.text, "html.parser")
text = soup.get_text()

# 分かち書き
tagger = MeCab.Tagger("-Owakati")
words = tagger.parse(text)

# 前処理したテキストを一旦ファイルに保存
with open("text.txt", "w", encoding="utf-8") as f:
    f.write(words)

# 訓練用データセットの設定
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer, file_path="text.txt", block_size=128
)

# データ入力の設定
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 訓練の設定
training_args = TrainingArguments(
    output_dir="./soseki",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
)

# Trainer インスタンスの作成
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# 訓練の実施
trainer.train()

# 訓練済みモデルの保存
trainer.save_model("./soseki_model")

# 文章生成のためのコード
input_ids = tokenizer.encode("吾輩は", return_tensors="pt")
sample_outputs = model.generate(
    input_ids,
    do_sample=True,  # 確率的にテキストを生成
    max_length=20,  # 生成する文章の最大長
    temperature=0.9,  # ランダム性の度合い（大きいほどランダム）
    top_k=50,
    top_p=0.95,
    num_return_sequences=10,  # 生成する文章の数
)

for i, sample_output in enumerate(sample_outputs):
    print("{}".format(tokenizer.decode(sample_output, skip_special_tokens=True)))
